# Raspberry Pi Setup

## Dependencies

1. Its advised to create a contained environment to install all dependencies as explained [here](https://github.com/EdjeElectronics/TensorFlow-Lite-Object-Detection-on-Android-and-Raspberry-Pi/blob/master/Raspberry_Pi_Guide.md#step-1b-download-this-repository-and-create-virtual-environment)
   1. `cd ~`
   2. `sudo pip3 install virtualenv`
   3. `python3 -m venv tflite1-env`
   4. `source ~/tflite1-env/bin/activate`
   5. Note you'll have to run the above command every time you open a new terminal to work on this project now, you can add it to `~/.bashrc` so its executed automatically every time you open a terminal with this command:
      `echo 'source ~/tflite1-env/bin/activate' >> ~/.bashrc`
   6. If in a new terminal you see (tflite1-env) instead of (base) behind the prompt, it's working
2. Following [*TensorFlow-Lite-Object-Detection-on-Android-and-Raspberry-Pi*](https://github.com/EdjeElectronics/TensorFlow-Lite-Object-Detection-on-Android-and-Raspberry-Pi)
   - For the time being, installing picamera2 is a hassel due to the transition from the legacy camera to the libcamera stack on the raspberry  
     To install it, we need to build several dependencies, these are in `./RaspberryPi/picamera2Setup/` in this repo  
     [following these instructions](https://www.raspberrypi.com/news/using-the-picamera2-library-with-tensorflow-lite/), we have to:
     1. Install dependencies:
        ```bash
          sudo apt update
          sudo apt install -y libboost-dev
          sudo apt install -y libgnutls28-dev openssl libtiff5-dev
          sudo apt install -y qtbase5-dev libqt5core5a libqt5gui5 libqt5widgets5
          sudo apt install -y meson
          sudo pip3 install pyyaml ply
          sudo pip3 install --upgrade meson
          sudo apt install -y libglib2.0-dev libgstreamer-plugins-base1.0-dev
          sudo pip3 install pyopengl
          sudo apt install python3-pyqt5
          sudo pip3 install opencv-python==4.4.0.46
          sudo apt install -y libatlas-base-dev
          pip3 install numpy
          sudo apt install build-essentials
          sudo apt install git
          sudo apt install python3-pip
          pip3 install tflite-runtime
          pip3 install opencv-python==4.4.0.46
          pip3 install pillow
          sudo pip3 install numpy --upgrade

          pip3 install pyqt5

          sudo apt-get install libyaml-dev
          sudo apt-get install libudev-dev
          sudo apt install -y cmake
          sudo apt-get install python-jinja2
          sudo apt-get install python-ply
          sudo pip3 install pyyaml
          sudo apt-get install python3-sphinx
          sudo apt-get install qttools5-dev-tools
          pip3 install python-prctl==1.6.1
          pip3 install simplejpeg
          pip3 install piexif
          pip3 install pidng
          pip3 install PyOpenGL
        ```
     2. you might have to [increase the swap](https://stackoverflow.com/questions/30887143/make-j-8-g-internal-compiler-error-killed-program-cc1plus) to build it successfully too. Check with `top` that you have not much less than 2Gb with RAM and SWAP combined
     3. Building Picamera2 library:
          1. Install all the dependencies in */RaspberryPi/picamera2Setup/picamera2/README.rst*
               - Something like (the command is likely to be depracated rapidaly)  
                 `sudo apt-get install -y ninja-build pkg-config libgnutls28-dev openssl libgnutls28-dev openssl libboost-dev libudev-dev python3-sphinx doxygen graphviz texlive-latex-extra libgstreamer1.0-dev libgstreamer-plugins-base1.0-dev libevent-dev qtbase5-dev libqt5core5a libqt5gui5 libqt5widgets5 qttools5-dev-tools libtiff-dev liblttng-ust-dev python3-jinja2 lttng-tools libevent-dev libexif-dev libjpeg-dev`
          2. `cd /RaspberryPi/picamera2Setup/libcamera`
          3. `meson build --buildtype=release -Dpipelines=raspberrypi -Dipas=raspberrypi -Dv4l2=true -Dgstreamer=enabled -Dtest=false -Dlc-compliance=disabled -Dcam=disabled -Dqcam=enabled -Ddocumentation=disabled -Dpycamera=enabled`
          4. `ninja -C build`
          5. `sudo ninja -C build install`
     4. Building kmsxx library:
          1. `cd /RaspberryPi/picamera2Setup/kmsxx`
          2. `git submodule update --init`
          3. `meson build`
          4. `ninja -C build`
     5. To make everything run, you will also have to set your `PYTHONPATH` environment variable. For example, we can add it to the `.bashrc` file:  
        alter the following line so it points to your dependencies in your system in `/RaspberryPi/picamera2Setup/`
        `export PYTHONPATH=/home/pi/repo/RaspberryPi/picamera2Setup/picamera2:/home/pi/repo/RaspberryPi/picamera2Setup/libcamera/build/src/py:/home/pi/repo/RaspberryPi/picamera2Setup/kmsxx/build/py:/home/pi/repo/RaspberryPi/picamera2Setup/python-v4l2`
- notes/other links:
   - [opencv with raspberry libcamera in C++](https://github.com/Qengineering/Libcamera-OpenCV-RPi-Bullseye-32OS)

## Test the installation

In [ ]:
#!/usr/bin/python3

# Copyright (c) 2022 Raspberry Pi Ltd
# Author: Alasdair Allan <alasdair@raspberrypi.com>
# SPDX-License-Identifier: BSD-3-Clause

# A TensorFlow Lite example for Picamera2 on Raspberry Pi OS Bullseye
#
# Install necessary dependences before starting,
#
# $ sudo apt update
# $ sudo apt install build-essentials
# $ sudo apt install libatlas-base-dev
# $ sudo apt install python3-pip
# $ pip3 install tflite-runtime
# $ pip3 install opencv-python==4.4.0.46
# $ pip3 install pillow
# $ pip3 install numpy
#
# and run from the command line,
#
# $ python3 real_time_with_labels.py --model mobilenet_v2.tflite --label coco_labels.txt

import tflite_runtime.interpreter as tflite

import sys
import os
import argparse

import cv2
import numpy as np
from PIL import Image
from PIL import ImageFont, ImageDraw

from picamera2 import Picamera2, Preview, MappedArray

#normalSize = (640, 480)
#lowresSize = (320, 240)
normalSize = (1280, 960)
lowresSize = (640, 480)

rectangles = []


def ReadLabelFile(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()
    ret = {}
    for i,line in enumerate(lines):
        ret[i] = line.strip()
    return ret


def DrawRectangles(request):
    with MappedArray(request, "main") as m:
        for rect in rectangles:
            print(rect)
            rect_start = (int(rect[0] * 2) - 5, int(rect[1] * 2) - 5)
            rect_end = (int(rect[2] * 2) + 5, int(rect[3] * 2) + 5)
            cv2.rectangle(m.array, rect_start, rect_end, (0, 255, 0, 0))
            if len(rect) == 5:
                text = rect[4]
                font = cv2.FONT_HERSHEY_SIMPLEX
                cv2.putText(m.array, text, (int(rect[0] * 2) + 10, int(rect[1] * 2) + 10), font, 1, (255, 255, 255), 2, cv2.LINE_AA)


def InferenceTensorFlow(image, model, output, label=None):
    global rectangles

    if label:
        labels = ReadLabelFile(label)
    else:
        labels = None

    interpreter = tflite.Interpreter(model_path=model, num_threads=4)
    interpreter.allocate_tensors()

    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    height = input_details[0]['shape'][1]
    width = input_details[0]['shape'][2]
    floating_model = False
    if input_details[0]['dtype'] == np.float32:
        floating_model = True

    rgb = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
    initial_h, initial_w, channels = rgb.shape

    picture = cv2.resize(rgb, (width, height))

    input_data = np.expand_dims(picture, axis=0)
    if floating_model:
        input_data = (np.float32(input_data) - 127.5) / 127.5

    interpreter.set_tensor(input_details[0]['index'], input_data)

    interpreter.invoke()

    '''
    detected_boxes = interpreter.get_tensor(output_details[0]['index'])
    detected_classes = interpreter.get_tensor(output_details[1]['index'])
    detected_scores = interpreter.get_tensor(output_details[2]['index'])
    num_boxes = interpreter.get_tensor(output_details[3]['index'])
    '''

    detected_scores = interpreter.get_tensor(output_details[0]['index'])
    detected_boxes = interpreter.get_tensor(output_details[1]['index'])
    num_boxes = interpreter.get_tensor(output_details[2]['index'])
    detected_classes = interpreter.get_tensor(output_details[3]['index'])

    rectangles = []

    for i in range(int(num_boxes)):
        top, left, bottom, right = detected_boxes[0][i]
        classId = int(detected_classes[0][i])
        score = detected_scores[0][i]
        if score > 0.02:
            xmin = left * initial_w
            ymin = bottom * initial_h
            xmax = right * initial_w
            ymax = top * initial_h
            box = [xmin, ymin, xmax, ymax]
            rectangles.append(box)
            if labels:
                print(labels[classId], 'score = ', score)
                rectangles[-1].append(labels[classId])
            else:
                print('score = ', score)


def main():
    parser = argparse.ArgumentParser()
    parser.add_argument('--model',default="" ,help='Path of the detection model.', required=True)
    parser.add_argument('--label', help='Path of the labels file.')
    parser.add_argument('--output', help='File path of the output image.')
    args = parser.parse_args()

    if (args.output):
        output_file = args.output
    else:
        output_file = 'out.jpg'

    if (args.label):
        label_file = args.label
    else:
        label_file = None

    picam2 = Picamera2()
    picam2.start_preview(Preview.QTGL)
    config = picam2.preview_configuration(main={"size": normalSize},
                                          lores={"size": lowresSize, "format": "YUV420"})
    picam2.configure(config)

    stride = picam2.stream_configuration("lores")["stride"]
    picam2.post_callback = DrawRectangles

    picam2.start()

    while True:
        buffer = picam2.capture_buffer("lores")
        grey = buffer[:stride * lowresSize[1]].reshape((lowresSize[1], stride))
        result = InferenceTensorFlow(grey, args.model, output_file, label_file)


if __name__ == '__main__':
    main()
